In [10]:
import pandas as pd

from functools import reduce

In [35]:
products_df = pd.read_csv('products.csv')
orders_df = pd.read_csv('orders.csv')
orders_products_prior_df = pd.read_csv('order_products__prior.csv')
orders_products_train_df = pd.read_csv('order_products__train.csv')

orders_products_df = pd.concat([orders_products_prior_df, orders_products_train_df])

orders_train_df = orders_df[orders_df['eval_set'].isin(['prior', 'train'])]
orders_test_df = orders_df[orders_df['eval_set'] == 'test']

users_orders_products_train_df = reduce(lambda l,r: pd.merge(l,r,on='order_id'), [order_products_df, orders_train_df])
users_orders_products_test_df = reduce(lambda l,r: pd.merge(l,r,on='order_id'), [order_products_df, orders_test_df])


users_orders_products_train_df.head()
#users_orders_products_train_df.head()
#users_orders_products_test_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,6,40462,1,0,22352,prior,4,1,12,30.0
1,6,15873,2,0,22352,prior,4,1,12,30.0
2,6,41897,3,0,22352,prior,4,1,12,30.0
3,8,23423,1,1,3107,prior,5,4,6,17.0
4,13,17330,1,0,45082,prior,2,6,17,1.0


In [46]:
def prepare_txn_set():
    single_item_set = set()
    txn_set = {}
    for index, row in users_orders_products_train_df.iterrows():
        order_id = row['order_id']
        product_id = row['product_id']
        if product_id not in single_item_set:
            single_item_set.add(product_id)
        if order_id not in txn_set:            
            txn_set[order_id] = []
        if product_id not in txn_set[order_id]:
            txn_set[order_id].append(product_id)
            txn_set[order_id] = sorted(txn_set[order_id])
    return single_item_set, txn_set

In [47]:
single_item_set, txn_set = prepare_txn_set()
freq_sets, rules = apriori(single_item_set, txn_set)

print(freq_sets)

NameError: name 'apriori' is not defined

In [407]:
min_support = 0.2
min_confidence = 0.5

def apriori(txn_set):
    support_dict = {}
    itemset_dict = get_all_single_item_count(txn_set)
    txn_size = len(txn_set)
    k = 2
    while len(itemset_dict) > 0:
        print('itemset_dict.1',itemset_dict)
        c_k = get_candidate_itemsets(itemset_dict)
        print('c_k.1',c_k)
        if len(c_k) != 0:            
            for order_id in txn_set:
                item_list = txn_set[order_id]
                c_t = subset(c_k, item_list)
                for c in c_t:
                    c_k[c] = c_k[c] + 1
        print('c_k.2',c_k)
        itemset_dict = {k: v for k, v in c_k.items() if v/txn_size >= min_support}
        support_dict = {**support_dict, **itemset_dict}
        print('itemset_dict.2',itemset_dict)
        k = k + 1
    return support_dict, apriori_genrules(support_dict)

def apriori_genrules(support_dict):
    rules_dict = {}
    for itemset_tuple in support_dict:
        itemset_list = list(itemset_tuple)
        itemset_size = len(itemset_list)
        if itemset_size >= 3:
            for i in range(itemset_size-1, 1, -1):
                lhs = tuple(itemset_list[0:i])
                rhs = tuple(itemset_list[i:itemset_size])
                conf = support_dict[itemset_tuple]/support_dict[lhs]
                if conf >= min_confidence:
                    rules_dict[str(lhs)+'->'+str(rhs)] = conf
                else:
                    break
    return rules_dict

def get_all_single_item_count(txn_set):
    itemset_count_dict = {}
    txn_size = len(txn_set)
    for order_id in txn_set:
        itemset = txn_set[order_id]
        for item in itemset:
            if item not in itemset_count_dict:
                itemset_count_dict[item] = 0
            itemset_count_dict[item] = itemset_count_dict[item] + 1
    return {(k,): v for k, v in itemset_count_dict.items() if v/txn_size >= min_support}

def get_candidate_itemsets(itemset_dict):
    c_set = {}
    for itemset1 in itemset_dict:
        for itemset2 in itemset_dict:
            if differ_only_by_last_element(list(itemset1), list(itemset2)):
                c = itemset1 + itemset2[len(itemset2)-1:len(itemset2)]
                c_set[tuple(sorted(c))] = 0
    return c_set
                
def differ_only_by_last_element(l1, l2):
    if len(l1) == 0 and len(l2) == 0:
        return True
    if len(l1) > len(l2):
        return False
    matches = True
    for i in range(0, len(l1)-1):
        matches = matches and (l1[i] == l2[i])
        if not matches:
            break
    matches = matches and (l1[len(l1)-1] != l2[len(l2)-1])
    return matches
    
class Node:
    def __init__(self, key):
        self.key = key
        self.data = []
        self.children = {}
        
    def addData(self, d):
        self.data.append(d)
    
    def addChild(self, childKey):
        if childKey not in self.children:
            self.children[childKey] = Node(childKey)
        return self.children[childKey]
    
    def getChild(self, childKey):
        if childKey in self.children:
            return self.children[childKey]
        return None
    
    def isLeafNode(self):
        return len(self.children) == 0
    
    def nodeToString(self, indent):
        s = indent+'key:'+(str(self.key) if self.key is not None else 'None')+'\n'
        data = '),('.join(','.join(str(t) for t in d) for d in self.data)
        s = s+indent+'data:[('+data+')]\n'
        s = s+indent+'isLeafNode:'+str(self.isLeafNode())+'\n'
        l = len(self.children)
        if l==0:
            s=s+indent+'children:{}\n'
        else:
            s=s+indent+'children:{\n'
            for (key,val) in self.children.items():
                s=s+val.nodeToString(indent+'  ')
            s=s+indent+'}\n'
        return s
    
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
         return self.nodeToString('')
        
def build_hash_tree(c_k):
    hash_tree = {}
    size = 0
    for itemset in c_k:
        size = len(itemset)
        keyset = []
        for item in itemset:
            keyset.append(item % size)
        keyset = tuple(keyset)
        if keyset not in hash_tree:
            hash_tree[keyset] = []
        hash_tree[keyset].append(itemset)
    root = Node(None)
    for keyset in hash_tree:
        n = root
        for key in keyset:
            n = n.addChild(key)
        for itemset in hash_tree[keyset]:
            n.addData(itemset)        
    return size, root
      
def hash_tree_contains_select_item_list(root, item_list_tuple, limit):
    node = root
    for i in range(0,len(item_list_tuple)):
        #print('node.1=',node)
        node = node.getChild(item_list_tuple[i]%limit)
        #print('node.2=',node)
        if node is None:
            return False
        elif node.isLeafNode():
            print('node=',node)
            if item_list_tuple in node.data:
                return True
    return False
    
def collect_subsets(root, item_list, item_size, idx, size, limit):
    print('=======================================================')
    print('collect_subsets.1', item_list, item_size, idx, size, limit)
    subset_list = []
    if size > 0:
        subset_list = subset_list + collect_subsets(root, item_list, item_size, idx+1, size-1, limit)
    print('collect_subsets.2', item_list, item_size, idx, size, limit)
    item_list_tuple = tuple(item_list[idx:idx+limit])
    print('item_list_tuple.1',item_list_tuple)
    if hash_tree_contains_select_item_list(root, item_list_tuple, limit):
        subset_list.append(item_list_tuple)
    for i in range(idx+limit,item_size):
        item_list_tuple = tuple(item_list[idx:idx+limit-1]+item_list[i:i+1])
        print('item_list_tuple.2',item_list_tuple)
        if hash_tree_contains_select_item_list(root, item_list_tuple, limit):
            subset_list.append(item_list_tuple)
    print('subset_list',subset_list)
    return subset_list
    
def subset(c_k, item_list):
    c_t = {}
    size, root = build_hash_tree(c_k)
    item_list = sorted(item_list)
    item_size = len(item_list)
    if(item_size >= size):
        subset_list = collect_subsets(root, item_list, item_size, 0, size, size)
        for subset in subset_list:
            c_t[tuple(sorted(subset))] = 0
    return c_t

In [409]:
txn_set = dict()
txn_set[1] = [1,3,2]
txn_set[2] = [4,1,3]
txn_set[3] = [3,4,2]
txn_set[4] = [3]
txn_set[5] = [2]
txn_set[6] = [1,3,2,5,6,7]
txn_set[7] = [4,1,3,6,7]
txn_set[8] = [3,4,2,5,7]
txn_set[9] = [3,7,5]
txn_set[10]= [2,5,3]

#itemset_dict = get_all_single_item_count(txn_set)

#print(itemset_dict)

#c_k = get_candidate_itemsets(itemset_dict)
#print(c_k)
#c_k = get_candidate_itemsets(c_k)

#print(c_k)

#print(build_hash_tree(c_k)[1])

#c_t = subset(c_k, txn_set[2])

#print(c_t)

support_dict, rules_dict = apriori(txn_set)

print('support_dict',support_dict)
print('rules_dict',rules_dict)


itemset_dict.1 {(1,): 4, (3,): 9, (2,): 6, (4,): 4, (5,): 4, (6,): 2, (7,): 4}
c_k.1 {(1, 3): 0, (1, 2): 0, (1, 4): 0, (1, 5): 0, (1, 6): 0, (1, 7): 0, (2, 3): 0, (3, 4): 0, (3, 5): 0, (3, 6): 0, (3, 7): 0, (2, 4): 0, (2, 5): 0, (2, 6): 0, (2, 7): 0, (4, 5): 0, (4, 6): 0, (4, 7): 0, (5, 6): 0, (5, 7): 0, (6, 7): 0}
collect_subsets.1 [1, 2, 3] 3 0 2 2
collect_subsets.1 [1, 2, 3] 3 1 1 2
collect_subsets.1 [1, 2, 3] 3 2 0 2
collect_subsets.2 [1, 2, 3] 3 2 0 2
item_list_tuple.1 (3,)
subset_list []
collect_subsets.2 [1, 2, 3] 3 1 1 2
item_list_tuple.1 (2, 3)
node= key:1
data:[(2,3),(2,5),(2,7),(4,5),(4,7),(6,7)]
isLeafNode:True
children:{}

subset_list [(2, 3)]
collect_subsets.2 [1, 2, 3] 3 0 2 2
item_list_tuple.1 (1, 2)
node= key:0
data:[(1,2),(1,4),(1,6),(3,4),(3,6),(5,6)]
isLeafNode:True
children:{}

item_list_tuple.2 (1, 3)
node= key:1
data:[(1,3),(1,5),(1,7),(3,5),(3,7),(5,7)]
isLeafNode:True
children:{}

subset_list [(2, 3), (1, 2), (1, 3)]
collect_subsets.1 [1, 3, 4] 3 0 2 2
collect_